In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo_codes import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [9]:
# define file path and read raw data for nyc origin cbgs
path='/Users/Dara/Documents/Python/rp-safegraph/'
cbg_nyc = pd.read_csv(path +"nyc_cbg.csv")

In [10]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [11]:
#define the months and days
#monthList =["01","02","03","04","05","06", "07","08"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31] 
month = "04"
years = ["2019","2020"]
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [15]:
##Month Year v. Year Comparison
frames = []
for y in years:
    for i in range(0,8):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='bucketed_away_from_home_time', key_col_name='away_from_home_time', value_col_name='time_away_count')

            #Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str)
            df['orig_tract'] = df['orig_cbg'].str[:-1]
            df['orig_stco'] = df['orig_cbg'].str[:5]

            #Make new table with select columns
            df = df[['orig_tract','away_from_home_time','time_away_count']
            dff = pd.pivot_table(df,values=['time_away_count'],index=['orig_tract'],columns=['away_from_home_time'],aggfunc=np.sum,fill_value=0,margins=True)

            dff.to_excel(f'{path}output/{month}away/{y}-{month}{dayList[i]}.xlsx')
            
    mid = time.time()
    elapsed_mid = mid - start
    print(f'Mid run time - {elapsed_mid}')

SyntaxError: invalid syntax (<ipython-input-15-eeabaf387070>, line 22)

In [ ]:
end = time.time()
elapsed = start - end
print(f'Run time - {elapsed}')